Итоговая работа
=====================

Поздравляю вас, теперь вы вооружены знаниями по предобработке данных, и пора применить их на интересном кейсе.

Представлены данные такси, по которым мы должны предсказать возьмёт ли таксист заказ или нет(driver_response).

Принципы вы знаете, чек-лист аналогичен лабораторной работе.

Чек-лист:
1. Загрузите датасет taxi.csv.
2. Посмотрите на данные. Отобразите общую информацию по признакам (вспомните о describe и info). Напишите в markdown свои наблюдения.
3. Выявите пропуски, а также возможные причины их возникновения. Решите, что следует сделать с ними. Напишите в markdown свои наблюдения.
4. Оцените зависимости переменных между собой. Используйте корреляции. Будет хорошо, если воспользуетесь profile_report. Напишите в markdown свои наблюдения.
5. Определите стратегию преобразования категориальных признаков (т.е. как их сделать адекватными для моделей).
6. Найдите признаки, которые можно разделить на другие, или преобразовать в другой тип данных. Удалите лишние, при необходимости.
7. Разделите выборку на обучаемую и тестовую.
8. Обучите модель. Напишите в markdown свои наблюдения по полученным результатам. Хорошие результаты дают классификаторы RandomForest и XGBoost

Если возникнут затруднения, то смотрите на материал практических занятий. Данного там должно хватить для выполнения всех пунктов. Желаю успеха!

In [141]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import GradientBoostingClassifier as gb
from sklearn.metrics import roc_auc_score as roc, log_loss

In [142]:
data = pd.read_csv("../data/taxi.csv")

In [143]:
data.head(10)

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response
0,1105373,5,20,6080,174182,55.818842,37.334562,55.814567,37.355010,-1.000,-1.000,Economy,private,0
1,759733,5,14,6080,358774,55.805342,37.515023,55.819329,37.466398,18.802,25.217,Standard,private,1
2,416977,6,14,6080,866260,55.813978,37.347688,55.814827,37.354074,6.747,9.800,Economy,private,0
3,889660,2,6,6080,163522,55.745922,37.421748,55.743469,37.431130,-1.000,-1.000,Economy,private,1
4,1120055,4,16,6080,506710,55.803578,37.521602,55.812559,37.527407,12.383,19.250,Economy,private,1
5,1058203,6,15,6080,360328,55.818963,37.361033,55.814827,37.354074,6.747,9.800,Economy,private,0
6,103326,2,11,6080,615584,55.753508,37.663742,55.757251,37.659064,-1.000,-1.000,Standard,business,1
7,128861,5,20,6080,254374,55.839318,37.248862,55.839134,37.304101,1.517,3.933,Economy,private,0
8,493006,5,20,6080,753429,55.818748,37.338240,55.835764,37.294710,6.222,15.433,Economy,private,0
9,953983,6,7,6080,533820,55.839155,37.251947,55.833198,37.312197,10.410,23.083,Economy,private,0


In [144]:
data.describe()

,offer_gk,weekday_key,hour_key,driver_gk,order_gk,driver_latitude,driver_longitude,origin_order_latitude,origin_order_longitude,distance_km,duration_min,driver_response
count,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,5.662189e+05,3.159860,13.827790,3446.376360,460867.01291,55.700824,37.492292,55.772440,37.541637,18.393518,23.134841,0.492480
std,3.259360e+05,2.054222,7.306496,2038.738677,266064.98953,2.135903,1.454640,0.665985,0.467758,41.129296,33.890105,0.499946
min,4.000000e+00,0.000000,0.000000,14.000000,18.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,2.855552e+05,1.000000,8.000000,1421.000000,229178.50000,55.708651,37.413497,55.708032,37.407117,-1.000000,-1.000000,0.000000
50%,5.656715e+05,3.000000,16.000000,3506.500000,462420.00000,55.772904,37.533664,55.772561,37.532537,10.411500,19.700000,0.000000
75%,8.487198e+05,5.000000,20.000000,5242.000000,690894.00000,55.960667,37.624113,55.962000,37.624358,30.665250,37.867000,1.000000
max,1.130355e+06,6.000000,23.000000,6806.000000,920344.00000,56.174097,38.280547,56.311121,38.243608,5234.283000,3958.283000,1.000000


In [145]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
offer_gk                  100000 non-null int64
weekday_key               100000 non-null int64
hour_key                  100000 non-null int64
driver_gk                 100000 non-null int64
order_gk                  100000 non-null int64
driver_latitude           100000 non-null float64
driver_longitude          100000 non-null float64
origin_order_latitude     100000 non-null float64
origin_order_longitude    100000 non-null float64
distance_km               100000 non-null float64
duration_min              100000 non-null float64
offer_class_group         100000 non-null object
ride_type_desc            100000 non-null object
driver_response           100000 non-null int64
dtypes: float64(6), int64(6), object(2)
memory usage: 10.7+ MB


In [146]:
def distance_between_driver(f1,f2,l1,l2):
    d=math.acos(np.sin(f1)*np.sin(f2)+np.cos(f1)*np.cos(f2)*np.cos(l1-l2))
    return d
    

Номер оффера явно не несет полезной информации, номер дня и час, а также все категориальный данные надо преобразовать в ванхот, остальные атрибуты надо преобразовать.
Попробуем использовать широту и долготу.
По формуле cos(d) = sin(φА)·sin(φB) + cos(φА)·cos(φB)·cos(λА − λB)  найдем расстояние L = d·R


<br> где φА и φB — широты, λА, λB — долготы данных пунктов, d — расстояние между пунктами, измеряемое в радианах длиной дуги большого круга земного шара.


In [147]:
data['sd']=data.apply(lambda x: distance_between_driver(x['driver_latitude'], x['origin_order_latitude'], x['driver_longitude'],x['origin_order_longitude']), axis =  1)

In [148]:
data.drop(['driver_latitude','origin_order_latitude','driver_longitude','origin_order_longitude','offer_gk','order_gk'],axis = 1, inplace = True)

In [149]:
data['offer_class_group'] = pd.Categorical(data['offer_class_group'])
df_g = pd.get_dummies(data['offer_class_group'], prefix = 'offer_class_group')
data = pd.concat([data, df_g], axis=1)
data.head()

,weekday_key,hour_key,driver_gk,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response,sd,offer_class_group_Delivery,offer_class_group_Economy,offer_class_group_Kids,offer_class_group_Premium,offer_class_group_Standard,offer_class_group_Test,offer_class_group_VIP,offer_class_group_VIP+,offer_class_group_XL
0,5,20,6080,-1.000,-1.000,Economy,private,0,0.015800,0,1,0,0,0,0,0,0,0
1,5,14,6080,18.802,25.217,Standard,private,1,0.038645,0,0,0,0,1,0,0,0,0
2,6,14,6080,6.747,9.800,Economy,private,0,0.004816,0,1,0,0,0,0,0,0,0
3,2,6,6080,-1.000,-1.000,Economy,private,1,0.006957,0,1,0,0,0,0,0,0,0
4,4,16,6080,12.383,19.250,Economy,private,1,0.009951,0,1,0,0,0,0,0,0,0


In [150]:
data['ride_type_desc'] = pd.Categorical(data['ride_type_desc'])
df_g = pd.get_dummies(data['ride_type_desc'], prefix = 'ride_type_desc')
data = pd.concat([data, df_g], axis=1)
data.head()

,weekday_key,hour_key,driver_gk,distance_km,duration_min,offer_class_group,ride_type_desc,driver_response,sd,offer_class_group_Delivery,...,offer_class_group_Premium,offer_class_group_Standard,offer_class_group_Test,offer_class_group_VIP,offer_class_group_VIP+,offer_class_group_XL,ride_type_desc_SMB,ride_type_desc_affiliate,ride_type_desc_business,ride_type_desc_private
0,5,20,6080,-1.000,-1.000,Economy,private,0,0.015800,0,...,0,0,0,0,0,0,0,0,0,1
1,5,14,6080,18.802,25.217,Standard,private,1,0.038645,0,...,0,1,0,0,0,0,0,0,0,1
2,6,14,6080,6.747,9.800,Economy,private,0,0.004816,0,...,0,0,0,0,0,0,0,0,0,1
3,2,6,6080,-1.000,-1.000,Economy,private,1,0.006957,0,...,0,0,0,0,0,0,0,0,0,1
4,4,16,6080,12.383,19.250,Economy,private,1,0.009951,0,...,0,0,0,0,0,0,0,0,0,1


In [151]:
data.drop(['offer_class_group','ride_type_desc'], axis=1, inplace=True)

In [152]:

df=data.copy()
df.drop(['driver_response'], axis=1,inplace=True)

In [156]:
X_train, X_test, y_train, y_test = tts(df.values,
                                                     data.iloc[:,5:6].applymap(lambda x: int(x) if isinstance(x, float) else x).values,
                                                     train_size = 0.8,
                                                     random_state = 42,
                                                     stratify = data.iloc[:,0].values)

/home/david/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [166]:
from time import time
catt0 = time()
model = gb(random_state = 42)
model.fit(X_train, y_train)
log_loss(y_test, model.predict_proba(X_test)[:,1])
cat_trainingtime = time()-catt0
print(f'Секунд {cat_trainingtime} или Минут {cat_trainingtime/60}')

/home/david/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Секунд 13.109291553497314 или Минут 0.21848819255828858


In [167]:
print(roc(y_test, model.predict_proba(X_test)[:,1]))

0.8234954982987621
